# Hyperbolic Curve Fitting

This Python notebook can be used to fit a hyperbolic curve on any given data.

## Before You Start

Make sure that you are working on a copy of this notebook. Instructions on how to do so can be found [here](https://scribehow.com/shared/Copying_and_Using_a_Python_Notebook_from_GitHub__nPUb96VpS8ibc9BMFKzq1A).

The data needs to be in a CSV file found in your Google Drive. Instructions on how to make a properly formatted CSV file can be found [here](https://scribehow.com/shared/Format_and_Make_a_CSV_File__FfVill-DSfG9-OUCN-Z3ug).

To use this notebook, simply run each section of code (hidden by default) by pressing the "Run cell" button on the left side of each section. Some sections require you to specify certain variables before running the corresponding code. These variables will be explained in detail below.

In [ ]:
# @title Notebook Setup {display-mode: "form"}

# Sets base path on Google Drive for CSV data.
from google.colab import drive
drive.mount('/content/drive')
base_path = '/content/drive/MyDrive/'

# Imports other necessary packages and functions.
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import numpy as np

### Variable Explanation

- **file_path**: Indicates the location and name of your CSV file. If your CSV file is in a folder in your Google Drive, add "folder-name/" (no quotation marks) in front of the CSV file name. If the CSV file is only shared with you, you need to make a copy or a shortcut of it in your own Drive. Note that this is case and whitespace sensitive.

- **x_col** and **y_col**: Indicates the column labels for the independent and dependent variables, respectively, in your CSV file. Note that this is case and whitespace sensitive.

- **x_label** and **y_label**: Indicates the labels for the x- and y-axis, respectively, on your graph.

In [ ]:
# @title Data Setup {display-mode: "form"}

'''
TODO: Change file_path to match the location and name of your CSV file.
  - If your CSV file is in a folder in your Google Drive, add "folder-name/" in front of the CSV file name.
      Example: "folder/file.csv" or "folder/subfolder/file.csv" (include the quotation marks)
  - The variable base_path directs the read_csv function to look for the CSV file in your Google Drive.
    If the file is only shared with you, you need to make a copy or a shortcut of it in your Drive.
'''
# Reads the CSV file with the data into a data frame (table).
file_path = "file.csv" # @param {type: "string"}
data = pd.read_csv(base_path + file_path)


'''
TODO: Change the column labels stored in x_col and y_col to match the column labels for the independent and
      dependent variables, respectively, in your CSV file.
'''
# Extracts the x and y data from the data frame.
x_col = "x_column" # @param {type: "string"}
y_col = "y_column" # @param {type: "string"}
hyperbolic_data = pd.concat([data[x_col], data[y_col]], axis = 1)

# Makes sure all data points are numerical and drops those that are not.
hyperbolic_data = hyperbolic_data.apply(pd.to_numeric, errors = "coerce")
hyperbolic_data = hyperbolic_data.dropna()

# Saves columns into a variable for easy referencing
x_data = hyperbolic_data[x_col]
y_data = hyperbolic_data[y_col]


'''
TODO: Change the labels for the axes of the graph.
'''
# Assigns the axes labels to variables.
x_label = "Independent Variable (units)" # @param {type: "string"}
y_label = "Dependent Variable (units)" # @param {type: "string"}

# Graphs a scatter plot of the data points.
plt.scatter(x_data, y_data)
plt.xlabel(x_label)
plt.ylabel(y_label)
plt.show()

## How to Fit Your Data

After running the section of code above, the resulting graph shows what your data looks like as is. Now, you will fit your data to the following equation:
$$y=\frac{a\times x}{b+x}$$

To help the computer find a more accurate equation to describe your data, you will provide initial guesses for what the values of *a* and *b* should be. In the section of code below, you will indicate your estimates based on the following descriptions:
- The value of *a* describes the y-value of the horizontal asymptote.
- The value of *b* describes the x-value at half of the horizontal asymptote's y-value.

This figure visualizes the above descriptions.

![](Figure.png)

Note: If you need to use exponents in Python (e.g., for writing scientific notation), use double asterisks (\*\*). For example, 2 x 10^-5 is written as 2 * (10\*\*-5) in Python.

### Variable Explanation

- **a_guess** and **b_guess**: Indicates your initial guesses for the *a* and *b* values, respectively, based on the scatter plot of your data and the descriptions above. You do not have to be precise with your guesses; just a rough estimate is okay.

In [ ]:
# @title Data Fitting {display-mode: "form"}

# Defines the function for a hyperbolic curve with the form y = (a * x) / (b + x).
def hyperbolic_func(x, a, b):
  return (a * x) / (b + x)


'''
TODO: Change the estimated parameters stored in the variables a_guess and b_guess based on the scatter plot
      of your data and the descriptions above. You do not have to be precise with your guesses; just a rough
      estimate is okay.
'''
# Finds the best parameters for the hyperbolic function using the initial guesses in param_guess.
# Returns the optimized parameters and their estimated covariance.
a_guess = -100 # @param {type: "number"}
b_guess = -100 # @param {type: "number"}

param_guess = [a_guess, b_guess]
param_optimized, param_covar = curve_fit(hyperbolic_func, x_data, y_data, param_guess)
a_optimized = param_optimized[0]
b_optimized = param_optimized[1]

# Generates predicted y values of the hyperbolic function using the specified range of x values.
# Allows the fitted function to be graphed separately from the experimental data.
x_pred = np.linspace(min(x_data), max(x_data), 100)
y_pred = hyperbolic_func(x_pred, a_optimized, b_optimized)

# Graphs the data with the fitted hyperbolic curve.
plt.scatter(x_data, y_data, label = "Collected Data")
plt.plot(x_pred, y_pred, color = "black", label = "Fitted Function")
plt.xlabel(x_label)
plt.ylabel(y_label)
plt.legend()
plt.show()

In [ ]:
# @title Results {display-mode: "form"}

# Determines the standard deviation errors of the optimized parameters.
param_std_error = np.sqrt(np.diag(param_covar))

# Prints the optimized parameters and the associated standard deviation errors.
print(f"The optimized value for a is {a_optimized}.")
print(f"The standard deviation error for a is {param_std_error[0]}.\n")
print(f"The optimized value for b is {b_optimized}.")
print(f"The standard deviation error for b is {param_std_error[1]}.\n")

# Determines the total sum of squares of the data.
total_sum_sq = np.sum((y_data - y_data.mean())**2)

# Determines the residual sum of squares of the data and its fit.
y_pred_data = hyperbolic_func(x_data, a_optimized, b_optimized)
resid_sum_sq = np.sum((y_data - y_pred_data)**2)

# Calculates and prints the R2 value describing the fit of the curve on the data.
r2 = 1 - (resid_sum_sq / total_sum_sq)
print(f"The R2 coefficient of the hyperbolic fit is {r2:.4f}.")